In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory('dataset',validation_split=0.2, subset="training",seed=123
                                                       ,image_size=(60, 60)
                                                       ,batch_size=32)


Found 902 files belonging to 2 classes.
Using 722 files for training.


In [3]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  'test',seed=123,
  image_size=(60, 60),batch_size=32)

Found 67 files belonging to 2 classes.


In [4]:
print(val_ds.class_names)
class_names =train_ds.class_names

['bike', 'car']


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
normalization_layer = layers.Rescaling(1./255)

In [7]:
num_classes = len(class_names)
print(num_classes)
img_height = 60
img_width = 60

2


In [8]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 60, 60, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 60, 60, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0

In [11]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
23/23 [==============================] - 4s 31ms/step - loss: 0.5863 - accuracy: 0.6731 - val_loss: 0.2455 - val_accuracy: 0.8955
Epoch 2/10
23/23 [==============================] - 0s 7ms/step - loss: 0.3096 - accuracy: 0.8684 - val_loss: 0.0148 - val_accuracy: 1.0000
Epoch 3/10
23/23 [==============================] - 0s 7ms/step - loss: 0.1969 - accuracy: 0.9321 - val_loss: 0.0083 - val_accuracy: 1.0000
Epoch 4/10
23/23 [==============================] - 0s 7ms/step - loss: 0.1432 - accuracy: 0.9501 - val_loss: 0.0847 - val_accuracy: 0.9701
Epoch 5/10
23/23 [==============================] - 0s 7ms/step - loss: 0.1206 - accuracy: 0.9515 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 6/10
23/23 [==============================] - 0s 7ms/step - loss: 0.0919 - accuracy: 0.9626 - val_loss: 0.0809 - val_accuracy: 0.9851
Epoch 7/10
23/23 [==============================] - 0s 7ms/step - loss: 0.0659 - accuracy: 0.9778 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 8/10
23/23 [=

In [16]:
import PIL
img = tf.keras.utils.load_img('byce.jfif'
    , target_size=(img_height, img_width))

In [17]:
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

In [18]:
import numpy as np
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to bike with a 99.39 percent confidence.
